In [ ]:
#******************************************************************************************************************************************************#
"""
    @author Shaela Khan, Created : 12th August, Last Updated 26th August, Friday 2022.

#  birdy2.ipynb  -> Experimentation
#  Bird Recognition from birdsongs using Deep learning ->s
#  DataSource : - https://www.kaggle.com/datasets/rtatman/british-birdsong-dataset  (Aggregated from the original Xeno-Canto Dataset.)
#  Provided dataset has a directory with - ./dataset/ --> Original audio dataset.
#                                        -   ./img_data2 --> audio files converted into image files --> contains spectrographs of audio data.
#
#
#  We then create a CNN model with possible usage of pre-trained models, that can identify the difference classes defined - this is a supervised learning
#  problem.
#  Input: birdsong_metadata.csv + songs
#  Output : T
"""
#*******************************************************************************************************************************************************#


In [51]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from librosa import core, onset, feature, display
import soundfile as sf
import umap
from IPython.display import Audio
import sklearn
import warnings
warnings.filterwarnings('ignore')

import os,sys,random,math
from random import seed, random, randint
from tqdm import tqdm
import pathlib
from pathlib import Path
# import import_ipynb,import nbimporter , import helper-functions


# FILE PROCESSING
import csv
import pandas as pd
import shutil
from shutil import move
from itertools import groupby

# Plotting AND Visuals.
import matplotlib.pyplot as plt
import numpy as np, shutil, itertools, pickle,seaborn as sn, math, time,scipy

# Stats
from scipy.spatial import distance
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import normalize
from mpl_toolkits.mplot3d import Axes3D
from scipy import spatial


# Image processing libraries
#!pip install Pillow
from PIL import Image


# EXTRA -
%matplotlib inline

# ML, statistics
import warnings
warnings.filterwarnings('ignore')
import scipy
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix, roc_curve, auc, roc_auc_score
from sklearn.decomposition import PCA
#from sklearn.preprocessing import OneHotEncoder


# TENSORFLOW LIBRARIES. BACK-END SUPPORT.
# Creating Model and model Evaluation
import tensorflow as tf
import tensorboard,tensorflow_estimator
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import layers

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation
from tensorflow.keras.layers import GlobalMaxPooling2D, GlobalAveragePooling1D, AveragePooling2D, Input, Add
from tensorflow.keras.optimizers import Adam,SGD,RMSprop
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint

import tensorflow.keras.utils
import tensorflow.keras.activations
import tensorflow.keras.applications
import tensorflow.keras.callbacks
import tensorflow.keras.initializers
from tensorflow.keras import losses,metrics


# Audio file processing
import librosa
import librosa.display
import soundfile as sf


# EXTRA EXTRA - because they don't work upstairs?
import pydub
from pydub import AudioSegment
import IPython
from IPython.display import Audio
warnings.filterwarnings('ignore')


# Start Code Here.
print("Hello World ! ")
print("********** !**********")
print("\n Tensorflow version: ", tf.__version__)


print("Hello Experiment.")

# Helper function- to be moved to helper-functions.ipynb
from datetime import datetime
def timer():
    # datetime object containing current date and time
    now = datetime.now()
    print("now =", now)
    # dd/mm/YY H:M:S
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    print("date and time =", dt_string)

print("\n")

Hello World ! 
********** !**********

 Tensorflow version:  2.2.0
Hello Experiment.




In [ ]:
genres = "northern cardinal,northern flicker,northern mockingbird,northern parula,northern pintail,northern roughwinged swallow,northern shoveler,northern waterthrush,nuttells woodpecker,olive sided flycatcher,orange crowned warbler,peregrine falcon,phainopepla,pied billed grebe,rose breasted grosbeak,snowgoose,sora,stellers jay,wild turkey, yellow billed flycatcher".split(',')
#print(genres)


# CONVERTING BIRD SONGS WHICH ARE AUDIO FILES INTO IMAGE FILES- FOR TEST DATA --> iMAGES ARE SPECTROGRAPHS OF THE AUDIO DATA.

cmap = plt.get_cmap('inferno')
plt.figure(figsize=(8,8))
#pathlib.Path('img_data2/').mkdir(parents=True, exist_ok=True)

for g in genres:
    pathlib.Path('img_data2/'+g).mkdir(parents=True, exist_ok=True)

    for filename in os.listdir('./dataset/'+g+'/'):
        songname = './dataset/'+g+'/'+filename
        y, sr = librosa.load(songname, mono=True, duration=5)
        print(y.shape)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off')
        base = os.path.splitext(filename)[0]
        # os.rename(my_file, base + '.bin')
        plt.savefig('img_data2/'+g+'/'+base+'.png')
        #plt.savefig('img_data2/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()



In [ ]:
# Creating data generators.
# rescale all pixel values from 0-255, so after this step all our pixel values are in range (0,1)#train_df['labels'] = train_df['english_cname'].astype(str)

print("Create DATA generators for the model.\n")

datagen=ImageDataGenerator(rescale=1./255,validation_split=0.3) # Training and Validation split 70/30
#test_datagen = ImageDataGenerator(rescale=1./255)



train_set =datagen.flow_from_directory('./img_data2/',
                                       target_size=(224,224),batch_size=8,
                                       class_mode='categorical',subset="training",
                                       seed=42,
                                       shuffle = False)
print('Train generator created')

validation_set = datagen.flow_from_directory('./img_data2/',
                                             target_size=(224, 224),
                                             batch_size=8,class_mode='categorical',
                                             subset="validation",seed=42,
                                             shuffle = False )


print('Validation generator created')


# test_set =test_datagen.flow_from_directory(test_df,target_size=(64, 64),
                                          # batch_size=32, class_mode='categorical', shuffle = False)



# test_gen=tvgen.flow_from_dataframe(test_df, x_col='filepaths', y_col='labels',target_size=img_size,
                                  # class_mode='categorical', color_mode='rgb', shuffle=False, batch_size=2)
train_image ,train_labels = train_set.next()
val_image, val_labels = validation_set.next()


print("\n ImageGenerator created data train set    :  ",train_image.shape)
print("\n ImageGenerator created data validation set:  ",val_image.shape)
print("\n Sanity check Line.----------")



#print("Printing labels of the validation dataset: \n",val_generator.labels)
#print("Printing label- names  of the training dataset: \n",train_labels)
#print("Printing label- names  of the validation dataset: \n",val_labels)

In [ ]:

resnet50 = keras.applications.resnet50

# Model Creation
dropout_dense_layer = 0.3
conv_model = resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
for layer in conv_model.layers:
    layer.trainable = False
x = tf.keras.layers.Flatten()(conv_model.output)
x = tf.keras.layers.Dense(100, activation='relu')(x)
x = tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 )(x)
x = tf.keras.layers.Dense(100, activation='relu')(x)
x = tf.keras.layers.Dense(100, activation='relu')(x)
x = tf.keras.layers.Dropout(dropout_dense_layer)(x)
predictions = tf.keras.layers.Dense(20, activation='softmax')(x)
full_model = tf.keras.models.Model(inputs=conv_model.input, outputs=predictions)
full_model.summary()




In [ ]:
# Step : 2 Training and running the model.  RESNET50 ---> The right way??

rcallback = [ModelCheckpoint(filepath='./models/bestModel.hdf5', monitor='val_loss', save_best_only=True, mode='auto')]

full_model.compile(loss='categorical_crossentropy',
                  optimizer=keras.optimizers.Adam(lr=0.005),
                  metrics=['acc'])


history__ = full_model.fit(train_set,
                           validation_data = validation_set,
                           callbacks=rcallback,
                           epochs=50)
timer()
full_model.save_weights('resnet50.h5')

In [ ]:
#Declare variables required for prediction case.
classes_to_predict = genres
validation_df = pd.DataFrame()


# Create dataframe of the predictions --taking in fraction of the data slice.
preds = full_model.predict(validation_set)
validation_df = pd.DataFrame(columns=["prediction", "groundtruth", "correct_prediction"])


for pred, groundtruth in zip(preds[:20], validation_set.__getitem__(0)[1]):
    validation_df = validation_df.append({"prediction":classes_to_predict[np.argmax(pred)],
                                       "groundtruth":classes_to_predict[np.argmax(groundtruth)],
                                       "correct_prediction":np.argmax(pred)==np.argmax(groundtruth)}, ignore_index=True)

print("Predictions: \n")
print(validation_df)

In [2]:

print("Say something.")


Say something.
